# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902018


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:27,  3.03s/it]

Rendering models:  10%|▉         | 3/31 [00:06<01:05,  2.32s/it]

Rendering models:  16%|█▌        | 5/31 [00:09<00:52,  2.01s/it]

Rendering models:  19%|█▉        | 6/31 [00:10<00:44,  1.78s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:25<00:01,  1.59it/s]

Rendering models:  94%|█████████▎| 29/31 [00:25<00:01,  1.93it/s]

Rendering models:  97%|█████████▋| 30/31 [00:25<00:00,  1.78it/s]

Rendering models: 100%|██████████| 31/31 [00:26<00:00,  1.81it/s]

equidad1                               0.014041
not-logged-in-cae133427d99aac1e554     0.000548
elizabeth_s                            0.002096
not-logged-in-ea8ed61a7274cfb6a43b    15.573491
ZigZagoon999                           0.000896
Astronomy4U                            0.000801
StephAChamber                          0.000386
Thedunkmasta                           0.000920
ElisabethB                             0.007264
not-logged-in-89733ae083faff644c04     0.001301
not-logged-in-63bb10dafda97df3ea97     0.010302
not-logged-in-aee0c500567f027c8b38     0.011514
alexismfernandez                       0.000847
gabrielweiss                           0.002446
Planetme                               0.000879
Chrisle206                             0.000516
Lavadude                               0.042608
41Letters                              0.000272
ts135992                               0.000202
clars915                               0.000392
not-logged-in-cf8f3b280ec60eaf25b5     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())